# Система распознавания предметов интерьера в потоковом видео
## Часть 6: Сбор статистики по площади, занимаемой объектом на кадре
### Воропаев Роберт, 2020

В этой части проводится сбор статистики средней площади занимаемой объектами разных классов на изображениях датасета. Позже эта информация будет использована для определения порога показа маски.

In [65]:
class def_config:

    main_data_dir = "data/ADE20K_encoded/"
    callbacks_dir = "callbacks/"

    img_shape = 256
    classes_num = 17

    batch_size = 4
    epoch_num = 1
    train_coef = 0.001
    learning_rate = 0.0001

    last_activation = "sigmoid"
    loss_function = "categorical_crossentropy"

    gpu_memory_limit = 0.8
    cpu_threads_num = 2

    callbacks_monitor = "val_jaccard_coef"
    callbacks_data_format = "%m.%d_%H-%M"
    file_name = "DefName"
    
    is_load = False
    weight_path = None
    
    argparse_is_on = False
    
    
args = def_config()

In [66]:
%matplotlib inline

# System
import os
import sys
import datetime
import time

# Base
import numpy as np
import cv2
import random
from matplotlib import pyplot as plt

# Keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPool2D, UpSampling2D, Conv2DTranspose
from keras.layers import Dropout, BatchNormalization, Concatenate
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, CSVLogger

# Preprocessing
from keras.utils import Sequence, to_categorical
from keras.utils.vis_utils import plot_model

# Backend
import tensorflow as tf
from keras import backend as K
from tensorflow.python.client import device_lib

# Seed
seed = 99
np.random.seed(seed)
random.seed(seed)

from scripts_.PathFinder import PathFinder
from scripts_.SegEncoder import SegEncoder
from scripts_.ClassList import ClassList

pf = PathFinder()
cl = ClassList(load_class_encode=True, class_encode_path="static/class_encode.txt")

In [67]:
main_data_dir = args.main_data_dir

train_dir = main_data_dir + "train/"
val_dir = main_data_dir + "val/"

img_train_dir = train_dir + "img/"
mask_train_dir = train_dir + "mask/"

img_val_dir = val_dir + "img/"
mask_val_dir = val_dir + "mask/"

train_size = len(os.listdir(path=train_dir + "img/"))
val_size = len(os.listdir(path=val_dir + "img/"))

img_shape = args.img_shape
batch_size = args.batch_size
classes_num = args.classes_num

epoch_num = args.epoch_num
train_coef = args.train_coef
learning_rate = args.learning_rate

loss_function = args.loss_function
last_activation = args.last_activation

In [70]:
%%time
area_dict = {i : [] for i in range(0, classes_num)}
for file, path in pf.data_gen(mask_train_dir, return_path=True):
    train_mask = cv2.imread(path + "/" + file, cv2.IMREAD_GRAYSCALE)
    train_mask = cv2.resize(train_mask, (img_shape, img_shape), interpolation=cv2.INTER_NEAREST)
    train_mask = train_mask.reshape(img_shape, img_shape, 1)
    train_mask = to_categorical(train_mask, num_classes=classes_num)
    for channel in range(0, classes_num):
        if train_mask[:, :, channel].sum() != 0:
            area_dict[channel].append(train_mask[:, :, channel].sum() / img_shape**2)

CPU times: user 43.9 s, sys: 82.3 ms, total: 44 s
Wall time: 43.9 s


In [72]:
with open("static/class_areas.txt", "w") as f:
    for i in range(classes_num):
        f.write(str(i) + '\t' + str(np.average(np.array(area_dict[i]))) + "\t" + str(cl.class_list[i]) + "\n")